In [1]:
import torch
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

print(f"System Version: {sys.version}")
print(f"Pytorch Version: {torch.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy Version: {np.__version__}")

System Version: 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
Pytorch Version: 2.7.1+cpu
Pandas Version: 2.3.0
Numpy Version: 2.3.0


In [ ]:
class PlayingCardDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        super().__init__()
        self.data = ImageFolder(root=data_dir, transform=transform)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return super().__getitem__(index)
    
    @property
    def classes(self):
        return self.data.classes